# Notebook for End to End testing in the MID ITF

- Uses digitizers for the 2 inputs to the correlator
- Uses 4 FSPs (800 MHz)
- Receives and plots the visibilities to SDP

### Pipeline URLs
These pipeline jobs were used to deploy the [Telescope Control System in the Mid ITF](https://confluence.skatelescope.org/display/SE/Stakeholder+diagram+of+Mid+ITF+Control+Interface+and+deployed+software):
##### [Redeploy SUT](https://gitlab.com/ska-telescope/ska-mid-itf/-/jobs/7634476752)
##### [Redeploy SKA001](https://gitlab.com/ska-telescope/ska-mid-itf/-/jobs/7632605231)
##### [Redploy SKA036](https://gitlab.com/ska-telescope/ska-mid-itf/-/jobs/7632605253)


In [1]:
import os
from time import sleep, localtime, strftime, time
from ska_mid_jupyter_notebooks.helpers.configuration import get_band_frequency_limits, get_dish_namespace

print("Setting up parameters\n")

# Set manually
CLUSTER_DOMAIN = "miditf.internal.skao.int"
SUT_NAMESPACE = "ci-ska-mid-itf-at-2249-e2e-tmc-bdd-test"
RECEPTORS = ["SKA001", "SKA036"]
SCAN_BAND = 1 # Select band
SCAN_DURATION = 10.0 # Determines how long the Scan cell will be kept active

# Get dish namespaces
SKA001_NAMESPACE = get_dish_namespace(SUT_NAMESPACE, "SKA001")
SKA036_NAMESPACE = get_dish_namespace(SUT_NAMESPACE, "SKA036")

TANGO_HOST = f"tango-databaseds.{SUT_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

SKA001_TANGO_HOST = f"tango-databaseds.{SKA001_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
SKA036_TANGO_HOST = f"tango-databaseds.{SKA036_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"

sut_taranta_devices_url = (
    f"https://k8s.{CLUSTER_DOMAIN}/{SUT_NAMESPACE}/taranta/devices"
)

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

KAFKA_PORT = 9092
KAFKA_SERVICE_NAME = "ska-sdp-kafka"
KAFKA_ENDPOINT = (
    f"{KAFKA_SERVICE_NAME}.{SUT_NAMESPACE}.svc.{CLUSTER_DOMAIN}:{KAFKA_PORT}"
)

print("Links for Taranta and QA Display")

print("System Under Test")
print(f"https://k8s.{CLUSTER_DOMAIN}/{SUT_NAMESPACE}/taranta/devices")

print("QA Display")
print(f"https://k8s.{CLUSTER_DOMAIN}/{SUT_NAMESPACE}/signal/display/")
print("\n")
print("Dish LMC links")
print(f"https://k8s.{CLUSTER_DOMAIN}/{SKA001_NAMESPACE}/taranta/devices")
print(f"https://k8s.{CLUSTER_DOMAIN}/{SKA036_NAMESPACE}/taranta/devices")


Setting up parameters

Links for Taranta and QA Display
System Under Test
https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-2249-e2e-tmc-bdd-test/taranta/devices
QA Display
https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-2249-e2e-tmc-bdd-test/signal/display/


Dish LMC links
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska001-at-2249-e2e-tmc-bdd-test/taranta/devices
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska036-at-2249-e2e-tmc-bdd-test/taranta/devices


### Set Up and Configure Device Proxies 

In [2]:
import json
from tango import DeviceProxy, EventType, DevState
from ska_control_model import ObsState
from ska_mid_jupyter_notebooks.dish.dish import DishMode
from notebook_tools.wait_for_tango import wait_for_event

print("Setting up the device proxies")

# TMC proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
cbf_fspcorrsubarray = DeviceProxy("mid_csp_cbf/fspcorrsubarray/01_01")

# Dish Leaf proxies
dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")
dish_leaf_node_ska036 = DeviceProxy("ska_mid/tm_leaf_node/d0036")

# SDP proxies
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

# Leaf node proxies
csp_subarray_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
sdp_subarray_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/sdp_subarray01")
csp_master_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/csp_master")

# Dish manager proxies
dish_manager_ska001 = DeviceProxy(f"{SKA001_TANGO_HOST}/mid-dish/dish-manager/ska001")
dish_manager_ska036 = DeviceProxy(f"{SKA036_TANGO_HOST}/mid-dish/dish-manager/ska036")

sleep(2)
print("\nChecking admin mode after setting to ONLINE (0):")
print("  CSP Control: ", csp_control.adminMode)
print("  CSP Subarray: ", csp_subarray.adminMode)
print("  CBF Controller: ", cbf_controller.adminMode)
print("  CBF Subarray: ", cbf_subarray.adminMode)

# Set CBF Simulation mode to false and CBF and CSP subarray command timeouts to 99s
csp_control.cbfSimulationMode = 1
csp_control.commandTimeout = 99
csp_subarray.commandTimeout = 99

sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

print("\nOK!")

Setting up the device proxies

Checking admin mode after setting to ONLINE (0):
  CSP Control:  adminMode.ONLINE
  CSP Subarray:  adminMode.ONLINE
  CBF Controller:  adminMode.ONLINE
  CBF Subarray:  adminMode.ONLINE

Checking CBF Simulation Mode and CBF Timeout:
  CBF Simulation Mode: True
  CBF Timeout: 99 sec

OK!


### Export System Configuration

In [3]:
import sys

from ska_mid_jupyter_notebooks.helpers.path import project_root

sys.path.append("../../src")
import pathlib
from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment
from typing import List
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa

test_equipment = TangoTestEquipment()
dish_deployments: List[TangoDishDeployment] = []
dishlmc_enabled = True
dish_ids = ["001", "036"]
branch_name = sut_taranta_devices_url
executon_environment = Environment.Staging
sut_namespace_override = ""
subarray_id = 1
sut = TangoSUTDeployment(
    branch_name,
    executon_environment,
    namespace_override=sut_namespace_override,
    subarray_index=subarray_id,
)
dish_namespace_overrides = ["", ""]
if dishlmc_enabled:
    for i, d in enumerate(dish_ids):
        dish = TangoDishDeployment(
            f"ska{d}",
            branch_name=branch_name,
            environment=executon_environment,
            namespace_override=dish_namespace_overrides[i],
        )
        print(f"Dish {d} configured: {dish}")
        dish_deployments.append(dish)
deployment: TangoDeployment
timestr = strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(),
    f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}",
)
os.makedirs(notebook_output_dir, exist_ok=True)
for deployment in [sut, test_equipment, *dish_deployments]:
    deployment.export_chart_configuration(output_dir=notebook_output_dir)

Dish 001 configured: TangoDishDeployment{dish_id=ska001; namespace=staging-dish-lmc-ska001; tango_host=tango-databaseds.staging-dish-lmc-ska001.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.staging-dish-lmc-ska001.svc.miditf.internal.skao.int:8765}
Dish 036 configured: TangoDishDeployment{dish_id=ska036; namespace=staging-dish-lmc-ska036; tango_host=tango-databaseds.staging-dish-lmc-ska036.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.staging-dish-lmc-ska036.svc.miditf.internal.skao.int:8765}
Exporting configuration using http://config-inspector.staging.svc.miditf.internal.skao.int:8765
ReleaseResponse (staging): {
    "chart": "ska-mid-itf-sut",
    "version": "23.5.0",
    "sub_charts": [
        {
            "chart": "ska-tango-base",
            "version": "0.4.10"
        },
        {
            "chart": "ska-tango-util",
            "version": "0.4.11"
 

### Load the Dish Vcc Config / Init Sys Params
#### [TMC Monitoring Taranta Dashboard](https://k8s.miditf.internal.skao.int/staging/taranta/dashboard?id=65dc5298b20f1600120e7a85&mode=run)
If `Dish VCC Config Set?` on the CentralNode is not `true`, run this cell.

In [4]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = (
    "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"
)

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
wait_for_event(
    tmc_central_node,
    "isDishVccConfigSet",
    True,
)

print("Dish VCC config has been set")

print("NB: Dish VCC config may not be set. Check manually.")

dish_config_json file contents: 
{'interface': 'https://schema.skao.int/ska-mid-cbf-initsysparam/1.0', 'tm_data_sources': ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata'], 'tm_data_filepath': 'instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json'}
Device ska_mid/tm_central/central_node attribute isDishVccConfigSet changedto the following desired value: True
Dish VCC config has been set
NB: Dish VCC config may not be set. Check manually.


#### Checking Config
If all `k` values are set to 1 (ITF specific test scenario), we are good to go - run this cell to check.

In [5]:
print(
    f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}"
)
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

TMC CSP Master's Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}

TMC CSP Master's Source Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"], "tm_data_filepath": "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"}


~~#### Side-step the hackery~~
~~This step was introduced after the Dark Night rose again. Pipelines were broken and we accidentally did not keep our `pyproject.toml` file in lock-step with the Engineering Tools repo. Thanks @AJ!~~

In [6]:
# ec_deployer = DeviceProxy("mid_csp_cbf/ec/deployer")

In [7]:
# # Run only if suspecting that something went seriously sideways with the deployment
# ec_deployer.targetTalons = [1, 2, 3, 4]
# ec_deployer.generate_config_jsons()
# ec_deployer.set_timeout_millis(600000)
# ec_deployer.download_artifacts()
# ec_deployer.configure_db()
# ec_deployer.set_timeout_millis(3000)

### Turn the Telescope On
Known issues:
When turning on Telescope, talon DDR memory calibration can fail. An indicator of this is `mid_csp_cbf/fspcoorrsubarray` attribute `obsState` being equal to `FAULT`. To adress this:
* Manually power cycle the PDUs to the talon boards - flip outlets 12,13,17,18 on PDU.
* Run `SetStandbyLPMode()` on both `DishLeafnode`s (see Contingencies)
* Rerun of the Telescope ON command.

In [8]:
assert cbf_fspcorrsubarray.obsstate == 2

print("Running the TelescopeOn command")

tmc_central_node.TelescopeOn()
wait_for_event(tmc_central_node, "telescopeState", DevState.ON)

print("YAY Telescope is ON")

Running the TelescopeOn command
Device ska_mid/tm_central/central_node attribute telescopeState changedto the following desired value: ON
YAY Telescope is ON


In [9]:
print("Verifying the states:")
print("  TMC Central Node State: ", tmc_central_node.State())
print("  CSP Control State: ", csp_control.State())
print("  CBF Controller State: ", cbf_controller.State())
print("  TMC Subarray State: ", tmc_subarray.State())

Verifying the states:
  TMC Central Node State:  ON
  CSP Control State:  ON
  CBF Controller State:  ON
  TMC Subarray State:  ON


### Assign Resources
Check that that the `proc-pb-...-XXXXXX-YYYYYY-vis-receive-KK-J` pods comes up in the staging-sdp namespace once this step completes successfully.

> **Known issue**: Sometimes the call to `sdp_subarray.state()` fails for some reason. Just rerun this command.

In [10]:
f_limits = get_band_frequency_limits(SCAN_BAND)

os.environ["TZ"] = "Africa/Johannesburg"
time_now = localtime()
date = strftime("%Y%m%d", time_now)
time_now = strftime("%H%M%S", time_now)
eb_id = f"eb-test-{date}-{time_now}"
pb_id = f"pb-test-{date}-{time_now}"

print(f"SDP Subarray state: {sdp_subarray.state()}")

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS
    assign_resources_json["sdp"]["processing_blocks"][0]["parameters"][
        "queue_connector_configuration"
    ]["exchanges"][0]["source"]["servers"] = KAFKA_ENDPOINT
    assign_resources_json["sdp"]["processing_blocks"][0]["parameters"][
        "extra_helm_values"
    ]["receiver"]["options"]["reception"][
        "stats_receiver_kafka_config"
    ] = f"{KAFKA_ENDPOINT}:json_workflow_state"
    assign_resources_json["sdp"]["execution_block"]["eb_id"] = eb_id
    assign_resources_json["sdp"]["processing_blocks"][0]["pb_id"] = pb_id
    assign_resources_json["sdp"]["execution_block"]["channels"][0]["spectral_windows"][0]["freq_min"] = f_limits["freq_min"]
    assign_resources_json["sdp"]["execution_block"]["channels"][0]["spectral_windows"][0]["freq_max"] = f_limits["freq_max"]

print(f"PB ID: {pb_id}, EB ID: {eb_id}")
print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))

wait_for_event(cbf_subarray, "obsState", ObsState.IDLE)
wait_for_event(sdp_subarray_leaf_node, "sdpSubarrayObsState", ObsState.IDLE)
wait_for_event(csp_subarray_leaf_node, "cspSubarrayObsState", ObsState.IDLE)
wait_for_event(tmc_subarray, "obsState", ObsState.IDLE)

print("Ready to configure scan")

SDP Subarray state: ON
Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned
PB ID: pb-test-20240918-173230, EB ID: eb-test-20240918-173230

assign_resources_json file contents: 
{'interface': 'https://schema.skao.int/ska-tmc-assignresources/2.1', 'transaction_id': 'txn-....-00001', 'subarray_id': 1, 'dish': {'receptor_ids': ['SKA001', 'SKA036']}, 'sdp': {'interface': 'https://schema.skao.int/ska-sdp-assignres/0.4', 'resources': {'csp_links': [1, 2, 3, 4], 'receptors': ['SKA001', 'SKA036'], 'receive_nodes': 1}, 'execution_block': {'eb_id': 'eb-test-20240918-173230', 'context': {}, 'max_length': 21600.0, 'channels': [{'channels_id': 'vis_channels', 'spectral_windows': [{'spectral_window_id': 'fsp_1_channels', 'count': 59520, 'start': 0, 'stride': 1, 'freq_min': 296400000.0, 'freq_max': 1090900000.0, 'link_map': [[0, 0], [200, 1], [744, 2], [944, 3]]}]}], 'polarisations': [{'polarisations_id': 'all', 'corr_type': ['XX', 'XY', 'YX', '

### Optional: Check obstates IDLE

In [11]:
print(f"Long running command result: {tmc_subarray.longRunningCommandResult}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

assert (
    (tmc_subarray.obsState == 2)
    and (sdp_subarray_leaf_node.sdpSubarrayObsState == 2)
    and (csp_subarray_leaf_node.cspSubarrayObsState == 2)
    and (cbf_subarray.obsState == 2)
), f"TMC Subarray Observation State: {tmc_subarray.obsState} \n \
    CBF Subarray Observation State: {cbf_subarray.obsState} \n \
    SDP Subarray Leaf Node Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState} \n \
    CSP Subarray Leaf Node Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}"

print("All obsstate values IDLE!")

Long running command result: ('1726680750.7901094_94161200160398_AssignResources', '[0, "Command Completed"]')
CBF Subarray Receptors : ('SKA001', 'SKA036')
All obsstate values IDLE!


### Contingencies
These are methods that may be used to get the Telescope or subsystems out of unwanted states during a test run.

In [12]:
# # Take csp subarray leaf node back to EMPTY
# csp_subarray_leaf_node.Restart()

# # Bringing sdp subarray leaf node from RESOURCING to EMPTY
# sdp_subarray_leaf_node.ReleaseAllResources()

# # Bring dishes back to StandbyLPMode
# dish_leaf_node_ska001.SetStandbyLPMode()
# dish_leaf_node_ska036.SetStandbyLPMode()

# # Subarray node stuck in RESOURCING, CONFIGURING
# tmc_subarray.Abort()
# tmc_subarray.Restart()

# # CSP and SDP subarray leaf nodes in IDLE, but need to get back to EMPTY
# csp_subarray_leaf_node.ReleaseAllResources()
# sdp_subarray_leaf_node.ReleaseAllResources()

# # ON failed on CSP controller
# # # csp_control.On([])
# # csp_master_leaf_node_dp.On()

# # Configure scan contingencies
# # SDP Subaarray leaf node stuck in Configuring
# sdp_subarray_leaf_node.End()
# dish_leaf_node_ska001.TrackStop()
# dish_leaf_node_ska036.TrackStop()

# # Configure scan on CSP subarray leaf node directly
# temp_configure_scan_json_path = f""
# with open(temp_configure_scan_json_path, "r", encoding="utf-8") as json_data:
#     d = json.load(json_data)
#     csp_subarray_leaf_node.Configure(json.dumps(d))

# # Other
# tmc_central_node.TelescopeOff()
# tmc_subarray.Restart()

# Dishes in OPERATE, get them back to STANDBY_FP
# dish_manager_ska001.AbortCommands()
# dish_manager_ska001.setstandbylpmode()
# dish_manager_ska036.AbortCommands()
# dish_manager_ska036.setstandbylpmode()

# # IF DishManager goes into StandbyFP at any stage after Configure(), run SetOperateMode() on the DishLeafNode:
# dish_leaf_node_ska001.SetOperateMode()
# dish_leaf_node_ska036.SetOperateMode()

# # Manual Dish tracking commands (not necessarily working!) - SKB-513:
# dish_leaf_node_ska001.Track("")
# dish_leaf_node_ska036.Track("")

# # If SUT is in an intermediate state, first run:
# tmc_subarray.Abort()
# tmc_subarray.Restart()

# # and then manually get dishes to StandbyFP
# dish_leaf_node_ska001.SetStandbyFPMode()
# dish_leaf_node_ska036.SetStandbyFPMode()

### Work around to slew the dishes to near the start of track to avoid tracking timing errors

Could take +1min

#### UPDATE post TMC Workshop
No longer necessary, thanks to the Dish Simulator having been updated (and latest behaviour also followed by TMC DishLeafNodes).

In [13]:
# SKA001_TANGO_HOST = f"tango-databaseds.{SKA001_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
# dish_manager_ska001 = DeviceProxy(f"tango://{SKA001_TANGO_HOST}/mid-dish/dish-manager/ska001")
# SKA036_TANGO_HOST = f"tango-databaseds.{SKA036_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
# dish_manager_ska036 = DeviceProxy(f"tango://{SKA036_TANGO_HOST}/mid-dish/dish-manager/ska036")

# dish_manager_ska001.slew([181.0, 31.0])
# dish_manager_ska036.slew([181.0, 31.0])

# while (
#     dish_manager_ska001.achievedpointing[1] != 181.0
#     or dish_manager_ska001.achievedpointing[2] != 31.0
# ):
#     sleep(1)

# while (
#     dish_manager_ska036.achievedpointing[1] != 181.0
#     or dish_manager_ska036.achievedpointing[2] != 31.0
# ):
#     sleep(1)

# print("Done slewing")

### Configure Scan
Before running this cell, check:
1. Is the `PersistentVolumeClaim` present in the `-sdp` namespace?
2. Is the `vis-receive` pod running in the `-sdp` namespace (dependent on among others, the above prerequisite).


> **Subarray stuck in CONFIGURING**: 
> After running this cell, the SubarrayNode may remain stuck in `CONFIGURING` obsstate. The rest of the Notebook works around this issue by invoking commands directly on the SDP Subarray Leafnode. The issue is due to the CSP <> CBF interface not fully integrated using Long Running Commands yet, which is WIP at the moment.


In [14]:
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)
    configure_scan_json["dish"]["receiver_band"] = str(SCAN_BAND)
    configure_scan_json["csp"]["common"]["frequency_band"] = str(SCAN_BAND)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

tmc_subarray.Configure(json.dumps(configure_scan_json))


wait_for_event(csp_subarray_leaf_node, "cspSubarrayObsState", ObsState.READY)
wait_for_event(sdp_subarray_leaf_node, "sdpSubarrayObsState", ObsState.READY)
wait_for_event(dish_leaf_node_ska001, "dishMode", DishMode.OPERATE)
wait_for_event(dish_leaf_node_ska036, "dishMode", DishMode.OPERATE)
wait_for_event(tmc_subarray, "obsState", ObsState.READY)

print("Ready to scan")

Running the Configure command: subarray obsstate should go to Ready

configure_scan_json file contents: 
{'interface': 'https://schema.skao.int/ska-tmc-configure/2.0', 'pointing': {'target': {'reference_frame': 'ICRS', 'target_name': 'Polaris Australis', 'ra': '21:08:47.92', 'dec': '-88:57:22.9'}}, 'sdp': {'interface': 'https://schema.skao.int/ska-sdp-configure/0.4', 'scan_type': 'science'}, 'csp': {'interface': 'https://schema.skao.int/ska-csp-configure/2.4', 'subarray': {'subarray_name': 'Receptor(s)'}, 'common': {'config_id': '1 receptor, band 1, 2 FSP, no options', 'subarray_id': 1, 'frequency_band': '1'}, 'cbf': {'fsp': [{'fsp_id': 1, 'function_mode': 'CORR', 'frequency_slice_id': 1, 'zoom_factor': 0, 'integration_factor': 10, 'output_link_map': [[0, 1]], 'channel_offset': 0, 'zoom_window_tuning': 450000}, {'fsp_id': 2, 'function_mode': 'CORR', 'frequency_slice_id': 2, 'zoom_factor': 0, 'integration_factor': 10, 'output_link_map': [[0, 1]], 'channel_offset': 14880, 'zoom_window_tu

### Optional: Check obstates READY
This cell will fail unless the CSP Subarray Leafnode and SDP Subarray Leafnode are in `READY` observation state.

In [15]:
print("\nCBF Subarray Observation State: ", cbf_subarray.obsState)
print("SDP Subarray Observation State: ", sdp_subarray_leaf_node.sdpSubarrayObsState)
print("CSP Subarray Observation State: ", csp_subarray_leaf_node.cspSubarrayObsState)
assert (
    (cbf_subarray.obsState == 4)
    and (sdp_subarray_leaf_node.sdpSubarrayObsState == 4)
    and (csp_subarray_leaf_node.cspSubarrayObsState == 4)
)
print("OK to proceed: Run a Scan!")


CBF Subarray Observation State:  obsState.READY
SDP Subarray Observation State:  sdpSubarrayObsState.READY
CSP Subarray Observation State:  cspSubarrayObsState.READY
OK to proceed: Run a Scan!


### Scan
This worked for the first time on 2024/08/20, at the end of the Atlas/TMC Workshop!

In [16]:
print("Running the Scan command: subarray obsstate should go to Scanning")

with open(SCAN_FILE, encoding="utf-8") as f:
    scan_json = f.read()

print(f"\nscan_json file contents: \n{scan_json}")

tmc_subarray.Scan(scan_json)
wait_for_event(sdp_subarray_leaf_node, "sdpSubarrayObsState", ObsState.SCANNING)
wait_for_event(csp_subarray_leaf_node, "cspSubarrayObsState", ObsState.SCANNING)
wait_for_event(tmc_subarray, "obsState",ObsState.SCANNING)

print("Scan started successfully")
if type(SCAN_DURATION) is float:
    print(f"Keeping cell active for {SCAN_DURATION}s")
    time_start = time()
    while (time() - time_start) < SCAN_DURATION:
        sleep(1)
    
print("\nCBF Subarray Observation State: ", cbf_subarray.obsState)

Running the Scan command: subarray obsstate should go to Scanning

scan_json file contents: 
{
  "interface": "https://schema.skao.intg/ska-tmc-scan/2.0",
  "transaction_id": "txn-....-00001",
  "scan_id": 1
}
Device ska_mid/tm_leaf_node/sdp_subarray01 attribute sdpSubarrayObsState changedto the following desired value: 5
Device ska_mid/tm_leaf_node/csp_subarray01 attribute cspSubarrayObsState changedto the following desired value: 5
Device ska_mid/tm_subarray_node/1 attribute obsState changedto the following desired value: 5
Scan started successfully
Keeping cell active for 10.0s

CBF Subarray Observation State:  obsState.SCANNING


Check https://k8s.miditf.internal.skao.int/staging/signal/display/ to see output of SDP on the Signal Displays Dashboard! (`staging` namespace deployments only)

Check https://k8s.miditf.internal.skao.int/shared-ska-dataproducts/dashboard/ to see the Data Products being stored on the Data Product Dashboard!

### END SCAN


POSSIBLY NOT GOING TO WORK - CHECK DISHMODE FIRST! IF THE DISHES ARE IN StandbyFP FOR SOME REASON, GO TO THE CONTINGENCIES CELL

In [17]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

tmc_subarray.EndScan()

wait_for_event(sdp_subarray_leaf_node, "sdpSubarrayObsState",ObsState.READY)
wait_for_event(csp_subarray_leaf_node, "cspSubarrayObsState", ObsState.READY)
wait_for_event(tmc_subarray, "obsState", ObsState.READY)

print("Scan ended successfully")

print("\nTMC Subarray Observation State: ", tmc_subarray.obsState)

Running the End Scan command: subarray obsstate should go to Ready state
Device ska_mid/tm_leaf_node/sdp_subarray01 attribute sdpSubarrayObsState changedto the following desired value: 4
Device ska_mid/tm_leaf_node/csp_subarray01 attribute cspSubarrayObsState changedto the following desired value: 4
Device ska_mid/tm_subarray_node/1 attribute obsState changedto the following desired value: 4
Scan ended successfully

TMC Subarray Observation State:  obsState.READY


~~TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING~~

In [18]:
# print("Running the End Scan command: subarray obsstate should go to Ready state")

# sdp_subarray_leaf_node.EndScan()
# sleep(2)
# csp_subarray_leaf_node.EndScan()
# print("SDP Subarray Observation State: ",sdp_subarray_leaf_node.sdpSubarrayObsState)
# print("CSP Subarray Observation State: ",csp_subarray_leaf_node.cspSubarrayObsState)

### END

MIGHT NOT WORK - SEE EndScan() COMMENT ABOVE

In [19]:
print("Running the End command: subarray obsstate should go to Idle state")

tmc_subarray.End()

wait_for_event(sdp_subarray_leaf_node, "sdpSubarrayObsState", ObsState.IDLE)
wait_for_event(csp_subarray_leaf_node, "cspSubarrayObsState", ObsState.IDLE)
wait_for_event(tmc_subarray, "obsState", ObsState.IDLE)


print("Observation ended successfully")

print("\nTMC Subarray Observation State: ", cbf_subarray.obsState)

Running the End command: subarray obsstate should go to Idle state
Device ska_mid/tm_leaf_node/sdp_subarray01 attribute sdpSubarrayObsState changedto the following desired value: 2
Device ska_mid/tm_leaf_node/csp_subarray01 attribute cspSubarrayObsState changedto the following desired value: 2
Device ska_mid/tm_subarray_node/1 attribute obsState changedto the following desired value: 2
Observation ended successfully

TMC Subarray Observation State:  obsState.IDLE


~~TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING~~

In [20]:
# print("Running the End command: subarray obsstate should go to Idle state")

# sdp_subarray_leaf_node.End()
# sleep(2)
# csp_subarray_leaf_node.End()

# sleep(2)
# print("SDP Subarray Observation State: ",sdp_subarray_leaf_node.sdpSubarrayObsState)
# print("CSP Subarray Observation State: ",csp_subarray_leaf_node.cspSubarrayObsState)

### Release Resources

MIGHT NOT WORK - SEE ABOVE REGARDING `DishMode` GOING TO `StandbyFP` AND CONTINGENCIES

In [21]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

tmc_subarray.ReleaseAllResources()
wait_for_event(sdp_subarray_leaf_node, "sdpSubarrayObsState", ObsState.EMPTY)
wait_for_event(tmc_subarray, "obsState", ObsState.EMPTY, EventType.CHANGE_EVENT)

print("Resources released successfully")

print("TMC Subarray Observation State: ", tmc_subarray.obsState)

Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty
Device ska_mid/tm_leaf_node/sdp_subarray01 attribute sdpSubarrayObsState changedto the following desired value: 0
Device ska_mid/tm_subarray_node/1 attribute obsState changedto the following desired value: 0
Resources released successfully
TMC Subarray Observation State:  obsState.EMPTY


~~TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING~~

In [22]:
# print(
#     "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
# )

# sdp_subarray_leaf_node.ReleaseAllResources()
# sleep(2)
# csp_subarray_leaf_node.ReleaseAllResources()

# sleep(2)
# print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
# print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

### Check Telescope Status

In [23]:
print(f"TMC Central Node State: {tmc_central_node.state()}")
print("CSP Control State: ", csp_control.state())
print("CBF Controller State: ", cbf_controller.state())

TMC Central Node State: ON
CSP Control State:  ON
CBF Controller State:  ON


### Telescope Off

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

wait_for_event(tmc_central_node, "telescopeState", DevState.OFF)
wait_for_event(dish_leaf_node_ska001, "dishMode", DishMode.STANDBY_LP)
wait_for_event(dish_leaf_node_ska036, "dishMode", DishMode.STANDBY_LP)

print("TMC Central Node State: ", tmc_central_node.State())
print("CSP Control State: ", csp_control.State())
print("CBF Controller State: ", cbf_controller.State())

print("Telescope turned OFF")



## Configure Test Equipment

#### Section for making test equipment setup changes as required for some of the mid - aiv jama tests

In [ ]:
from bokeh.io import output_notebook

from ska_mid_jupyter_notebooks.test_equipment.rendering import (
    get_test_equipment_monitor_plot,
)
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

#### Print current status

In [ ]:
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

### Print Test Equipment Diagnostics

In [ ]:
# test_equipment.print_diagnostics()
tmc_subarray.Abort()

### Create Test Equipment Plot

In [ ]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(
    monitor_plot.handle_device_state_change
)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

Turn offline Test Equipment devices ONLINE

In [ ]:
# set any offline devices to online
test_equipment.turn_online()

#### Display Test Equipment Device States 

In [ ]:
test_equipment_state.state["devices_states"]

Band 1 setup

Set CW generator to approximately 480MHz (freq1) & -30dBm (power1), and wideband noise to approximately -132dBm/Hz at the SPF Band 1 Receiver inputs. ie. Band 1 output on and attenuation approx 10dB.  

Expected Result:  
*Test equipment configured*

In [ ]:
# test equipment setup values - band 1; use 950.000e6 for frequency for band 2

import time

frequency_to_set = 480.000e6
power_level = -30.0
attenuation = 10
band = 1

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(
    f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}"
)
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

# setup siggen
signal_generator.write_attribute("frequency", frequency_to_set)
signal_generator.write_attribute("power_dbm", power_level)
# setup SSC
SSC.write_attribute("Correlated_Noise_Source", True)
SSC.write_attribute("Uncorrelated_Noise_Sources", False)
SSC.write_attribute("Band", band)
# setup Attenuator
prog_atten.write_attribute("channel_1", attenuation)

sleep(3)
# print updated values and confirm updates
print(
    f"Updated signal generator frequency                  : {signal_generator.frequency}"
)
print(
    f"Updated signal generator power level                : {signal_generator.power_dbm}"
)
print(
    f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}"
)
print(
    f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}"
)
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}")

assert signal_generator.frequency == frequency_to_set, print(
    f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
)
assert signal_generator.power_dbm == power_level, print(
    f"Power level required is {power_level} but got {signal_generator.power_dbm}"
)
assert SSC.Correlated_Noise_Source, print(
    f"Correlated noise source required is {True} but got {SSC.Correlated_Noise_Source}"
)
assert not SSC.Uncorrelated_Noise_Sources, print(
    f"Uncorrelated noise source required is {False} but got {SSC.Uncorrelated_Noise_Sources}"
)
assert SSC.Band == band, print(f"Band required is {band} but got {SSC.Band}")
assert prog_atten.channel_1 == attenuation, print(
    f"Attenuation required is {attenuation} but got {prog_atten.channel_1}"
)

#### to print current setup only

In [ ]:
# print current only
signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

# print current
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(
    f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}"
)
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")

~10 seconds after recording has started, use Test Control Script to adjust CW generator signal level by -3dB. 

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [ ]:
sleep(
    10
)  # cell to be run in this notebook immediately after starting scn in rf-chain-and-channelisation_test_equipment notebook
power_level_minus_3_db = power_level - 3.0
signal_generator = test_equipment.signal_generator
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute("power_dbm", power_level_minus_3_db)
sleep(3)
print(f"Updated signal generator power level: {signal_generator.power_dbm}")

After a further ~10 seconds, use Test Control Script to adjust CW generator signal level by -3dB.  

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [ ]:
sleep(
    10
)  # cell to be run in this notebook immediately after starting scn in rf-chain-and-channelisation_test_equipment notebook
power_level_minus_6_db = power_level - 6.0
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute("power_dbm", power_level_minus_6_db)
sleep(3)
print(f"Updated signal generator power level: {signal_generator.power_dbm}")

After a further ~10seconds, use Test Control Script to increase the CW generator signal frequency by 10MHz.  

Expected Result:  
*CW frequency changes by 10MHz in SDP Spectrometer Display*

In [ ]:
sleep(10)
frequency_plus_10MHz = frequency_to_set + 10.0e6
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute("frequency", frequency_plus_10MHz)
sleep(3)
print(f"Updated signal generator frequency: {signal_generator.frequency}")